# Parse Nav.no sitemap

Denne notatboken analyserer
[nav.no/sitemap.xml](https://www.nav.no/sitemap.xml). Sitemap-en beskriver alle
åpne sider på [Nav.no](https://www.nav.no) og når de siste er oppdatert. Denne
informasjonen er fin å bruke når man skal laste ned innholdet.

In [ ]:
from rich import print

In [ ]:
# Filter for å fjerne stier som vi ikke er interessert i å indeksere
PATH_FILTER = ("/nav-og-samfunn/statistikk", "/nav-og-samfunn/kunnskap")

## Leser inn sitemap fra internett

In [ ]:
import httpx

raw_sitemap = httpx.get("https://www.nav.no/sitemap.xml").raise_for_status()

In [ ]:
import xml.etree.ElementTree as ET

sitemap = ET.fromstring(raw_sitemap.text)
print(f"Antall sider i sitemap: {len(sitemap)}")

## Filtrerer ut sider

In [ ]:
import datetime

from yarl import URL

to_index: dict[URL, datetime.datetime] = {}

for site in sitemap:
    if not any([path in site[0].text for path in PATH_FILTER]):
        url = URL(site[0].text)
        time = datetime.datetime.fromisoformat(site[1].text)
        to_index[url] = time
print(f"Antall sider etter filtrering: {len(to_index)}")

## Sjekk om siden er tilgjengelig

In [ ]:
from rich.progress import track

could_not_reach: list[URL] = []
reachable: list[URL] = []

with httpx.Client() as client:
    for site in track(to_index, description="Prøver å nå stier"):
        try:
            resp = client.get(str(site), follow_redirects=True).raise_for_status()
            reachable.append(URL(str(resp.url)))
        except httpx.HTTPStatusError:
            could_not_reach.append(site)

print(f"Antall sider som ikke kunne nåes: {len(could_not_reach)}")
print(f"Antall sider som vi klarte å nå: {len(reachable)}")

## Definere datastruktur

In [ ]:
from pydantic import BaseModel, HttpUrl


class SiteToIndex(BaseModel):
    """Representasjon av en nettside vi skal indeksere."""

    url: HttpUrl
    """URL til siden"""
    json_url: HttpUrl
    """URL til JSON objektet vi kan hente strukturert data"""
    last_modified: datetime.datetime | None
    """Når ble siden sist oppdatert"""

## Hent ut JSON data

In [ ]:
from rich.progress import track

index: list[SiteToIndex] = []

with httpx.Client() as client:
    for site in track(reachable, description="Aksesserer JSON"):
        resp = client.get(
            f"https://www.nav.no/_next/data/latest/{site.path}.json"
        ).raise_for_status()
        try:
            last_modified = resp.json()["pageProps"]["content"].get("modifiedTime")
            index.append(
                SiteToIndex(
                    url=str(site),
                    json_url=str(resp.request.url),
                    last_modified=last_modified,
                )
            )
        except KeyError:
            print(f"Fant ikke 'pageProps.content' for {site}")

print(f"Hentet JSON for {len(index)} sider")

## Skriv til lokal JSON

In [ ]:
import json

from rich.prompt import Prompt

if local_file := Prompt.ask("Filnavn å skrive til (tom avbryter skriving)"):
    if not local_file.endswith(".json"):
        local_file += ".json"
    with open(local_file, mode="w") as fil:
        json.dump([site.model_dump(mode="json") for site in index], fil)